In [38]:
import numpy as np
import random
import itertools
from collections import defaultdict
#CONTENT CATALOGUE
#number of content items
K=50
#content relativity matrix
#u_ij=np.zeros((K,K))
u_ij = np.random.rand(K, K)
#percentage of cached items
C=0.2*K
C=int(C)
#cache
cache=[0]*C
#content relevance threshold
u_min=0.5
#USER MODEL
#number of recomendations
N=2
#probability of ending viewing session
q=0.02
#probability of picking one item with equal prob. in case all are relevant
alpha=0.9
#probability when the user chooses from K items
p_k=1/K

actions=[]
actions_initialized=0
actions_num=[]


#state action space

def nested_dict():
    return defaultdict(nested_dict)
Space=nested_dict()


#def initialize_cache():
  #initialize cache
  #for i in range(0,C):
      #cache[i]=i+30

def initialize_cache():
   items_in_cache = random.sample(list(range(K)), C)
   for i in range(len(items_in_cache)):
    cache[i]=items_in_cache[i]


def initialize_relativity_matrix():
  #content relativity matrix initializations
  for i in range(K):
    for j in range(0, i):
        u_ij[i][j] = u_ij[j][i]
    u_ij[i][i] = 0



def item_cost(item):
  if item in cache:
    return 0
  else:
    return -1

#check if all items are above u_min
def all_bigger_than_u_min(current_item,recommended_items):
  for i in recommended_items:
    relevance=u_ij[i,current_item]
    if(relevance<u_min):
      return False
  return True



def initialize_state_action_space():
  for state in range(K):
    #print(state)
    items = list(range(K))
    other_items = items.copy()
    #other_items.remove(state)
    recommended_combinations = list(itertools.combinations(other_items, N))

    for item_set in recommended_combinations:
          item_set_str = [str(item) for item in item_set]  # Convert integers to strings
          action_name = '-'.join(item_set_str)
          action_outcomes_list=[]
          global actions_initialized
          if actions_initialized == 0:
            actions.append(action_name)
            actions_num.append(item_set)


          if(state in item_set):
            for i in other_items:
              action_outcomes_list.append((0,0,0,False))
          elif all_bigger_than_u_min(state,item_set):
            #print(action_name)
            #print(state)
            prob_of_picking_recommended=(1/N)*alpha*(1-q)
            prob_of_picking_random=p_k*(1-alpha)*(1-q)
            for i in other_items:
              if i in item_set:
                action_outcomes_list.append((prob_of_picking_recommended,i,item_cost(i),False))
              else:
                action_outcomes_list.append((prob_of_picking_random,i,item_cost(i),False))

            #print(state)
          else:
            for i in other_items:
              prob_of_picking_random=p_k*(1-q)
              action_outcomes_list.append((prob_of_picking_random,i,item_cost(i),False))

          action_outcomes_list.append((q,0,0,True))#with some probability end session
          Space[state][action_name]=action_outcomes_list
  actions_initialized=1
initialize_relativity_matrix()
initialize_cache()
  #print(cache)
initialize_state_action_space()
print(Space[6])


defaultdict(<function nested_dict at 0x7f6770227b50>, {'0-1': [(0.441, 0, -1, False), (0.441, 1, 0, False), (0.0019599999999999995, 2, -1, False), (0.0019599999999999995, 3, -1, False), (0.0019599999999999995, 4, -1, False), (0.0019599999999999995, 5, -1, False), (0.0019599999999999995, 6, -1, False), (0.0019599999999999995, 7, -1, False), (0.0019599999999999995, 8, -1, False), (0.0019599999999999995, 9, -1, False), (0.0019599999999999995, 10, -1, False), (0.0019599999999999995, 11, -1, False), (0.0019599999999999995, 12, -1, False), (0.0019599999999999995, 13, 0, False), (0.0019599999999999995, 14, -1, False), (0.0019599999999999995, 15, -1, False), (0.0019599999999999995, 16, -1, False), (0.0019599999999999995, 17, -1, False), (0.0019599999999999995, 18, -1, False), (0.0019599999999999995, 19, 0, False), (0.0019599999999999995, 20, -1, False), (0.0019599999999999995, 21, -1, False), (0.0019599999999999995, 22, 0, False), (0.0019599999999999995, 23, -1, False), (0.0019599999999999995,

In [39]:
import tkinter as tk #loads standard python GUI libraries
import numpy as np
import time
import random
import array


P=Space.copy()

# The next few lines are mostly for accounting
Tmax = 100000
size = len(P)


t = 0




#this one is generic to be applied in many AI gym compliant environments

def policy_evaluation(pi, P, gamma = 1.0, epsilon = 1e-10):  #inputs: (1) policy to be evaluated, (2) model of the environment (transition probabilities, etc., see previous cell), (3) discount factor (with default = 1), (4) convergence error (default = 10^{-10})
    t = 0   #there's more elegant ways to do this
    prev_V = np.zeros(len(P)) # use as "cost-to-go", i.e. for V(s')
    #print(len(P[0]))
    while True:
        V = np.zeros(len(P)) # current value function to be learnerd
        for s in range(len(P)):  # do for every state
            for prob, next_state, reward, done in P[s][pi(s)]:  # calculate one Bellman step --> i.e., sum over all probabilities of transitions and reward for that state, the action suggested by the (fixed) policy, the reward earned (dictated by the model), and the cost-to-go from the next state (which is also decided by the model)
                V[s] += prob * (reward + gamma * prev_V[next_state] * (not done))
        if np.max(np.abs(prev_V - V)) < epsilon: #check if the new V estimate is close enough to the previous one;
            break # if yes, finish loop
        prev_V = V.copy() #freeze the new values (to be used as the next V(s'))
        t += 1
    #print(len(P[0]))
    return V

def policy_improvement(V, P, gamma=1.0):  # takes a value function (as the cost to go V(s')), a model, and a discount parameter
    Q = np.zeros((len(P), len(P[0])), dtype=np.float64) #create a Q value array\
    action_index=[]
    for s in range(len(P)):        # for every state in the environment/model
        action_counter=0
        for a in list(P[0].keys()):  # and for every action in that state
            action_index.append(a)
            for prob, next_state, reward, done in P[s][a]:  #evaluate the action value based on the model and Value function given (which corresponds to the previous policy that we are trying to improve)
                Q[s][action_counter] += prob * (reward + gamma * V[next_state] * (not done))
                if s in actions_num[action_counter]:
                  Q[s][action_counter]=float('-inf')
            action_counter+=1
    new_pi_dict = {}
    Q_argmax = np.argmax(Q, axis=1)

    for s, a in enumerate(Q_argmax):
        new_pi_dict[s] = action_index[a]
        #print(action_index[a])

    new_pi = lambda s: new_pi_dict[s]  # this basically creates the new (improved) policy by choosing at each state s the action a that has the highest Q value (based on the Q array we just calculated)
    return new_pi

# policy iteration is simple, it will call alternatively policy evaluation then policy improvement, till the policy converges.

def policy_iteration(P, gamma = 1.0, epsilon = 1e-10):
    t = 0
    random_actions = np.random.choice(tuple(P[0].keys()), len(P))     # start with random actions for each state
    pi = lambda s: {s:a for s, a in enumerate(random_actions)}[s]     # and define your initial policy pi_0 based on these action (remember, we are passing policies around as python "functions", hence the need for this second line)

    while True:
        P_opt_dict = {s: pi(s) for s in range(K)}
        for key in P_opt_dict:
            item = P_opt_dict[key]
            print(f"Key: {key}, Item: {item}")
        old_pi = {s: pi(s) for s in range(len(P))}  #keep the old policy to compare with new
        V = policy_evaluation(pi,P,gamma,epsilon)   #evaluate latest policy --> you receive its converged value function        print(V)
        print(V)
        pi = policy_improvement(V,P,gamma)          #get a better policy using the value function of the previous one just calculated
        t += 1
        if old_pi == {s:pi(s) for s in range(len(P))}: # you have converged to the optimal policy if the "improved" policy is exactly the same as in the previous step
            break
    print('converged after %d iterations' %t) #keep track of the number of (outer) iterations to converge
    return V,pi





V_opt,P_opt = policy_iteration(P,0.5,0.001)   #just example of calling the various new functions we created.


print(V_opt)
P_opt_dict = {s: P_opt(s) for s in range(len(V_opt))}
for key in P_opt_dict:
    item = P_opt_dict[key]
    print(f"Key: {key}, Item: {item}")

    substring = item.split('-')[0]
    result = int(substring)
    print(u_ij[result,key])
    #for i in range (0,K):
      #print(u_ij[i][:])






Key: 0, Item: 7-27
Key: 1, Item: 32-40
Key: 2, Item: 22-24
Key: 3, Item: 1-7
Key: 4, Item: 8-16
Key: 5, Item: 17-21
Key: 6, Item: 4-10
Key: 7, Item: 0-32
Key: 8, Item: 16-24
Key: 9, Item: 42-47
Key: 10, Item: 29-41
Key: 11, Item: 13-29
Key: 12, Item: 6-19
Key: 13, Item: 7-41
Key: 14, Item: 7-28
Key: 15, Item: 12-47
Key: 16, Item: 18-42
Key: 17, Item: 9-14
Key: 18, Item: 19-41
Key: 19, Item: 13-49
Key: 20, Item: 10-26
Key: 21, Item: 40-43
Key: 22, Item: 23-41
Key: 23, Item: 22-37
Key: 24, Item: 36-45
Key: 25, Item: 28-41
Key: 26, Item: 11-29
Key: 27, Item: 15-21
Key: 28, Item: 22-27
Key: 29, Item: 10-29
Key: 30, Item: 14-48
Key: 31, Item: 37-46
Key: 32, Item: 18-43
Key: 33, Item: 30-41
Key: 34, Item: 10-37
Key: 35, Item: 13-44
Key: 36, Item: 0-27
Key: 37, Item: 12-15
Key: 38, Item: 14-42
Key: 39, Item: 38-40
Key: 40, Item: 1-7
Key: 41, Item: 8-19
Key: 42, Item: 8-38
Key: 43, Item: 11-17
Key: 44, Item: 4-48
Key: 45, Item: 4-10
Key: 46, Item: 11-22
Key: 47, Item: 15-19
Key: 48, Item: 28-3